### Decorator Application - Logger a Stacked decorator

In [133]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc) 
        result = fn(*args,**kwargs)
        print(f"{run_dt}: called {fn.__name__}")
        return result
    
    return inner
#pouze vypíše jméno a čas kdy byla spuštěná

In [134]:
@logged
def func_1():
    pass

In [135]:
@logged
def func_2():
    pass

In [136]:
func_1()

2020-06-02 11:18:06.360008+00:00: called func_1


In [137]:
func_2()

2020-06-02 11:18:16.196030+00:00: called func_2


Jasné, takto mohu dekorovat jakoukoliv funkci.

In [138]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print(f"{fn.__name__} ran for {end-start:.6f}s")
        return result
    return inner

In [142]:
@logged
@timed
def fact(n):
    from operator import mul #násobení
    from functools import reduce
    
    return reduce(mul, range(1,n+1))
#pěkný to zápis pro výpočet faktoriálu - chce to opáčko reduce funkce

In [143]:
fact(5)

fact ran for 0.000012s
2020-06-02 11:24:36.296362+00:00: called fact


120

Tadááá, naše krásná funkce může používat více dekorátorů.

In [144]:
fact = logged(timed(fact))

In [145]:
fact(5)

fact ran for 0.000008s
2020-06-02 11:25:21.323731+00:00: called fact
fact ran for 0.011246s
2020-06-02 11:25:21.323731+00:00: called fact


120

Zkusíme to trochu vyladit

In [147]:
def dec_1(fn):
    def inner():
        print("Running dec_1")
        return fn()
    return inner

In [148]:
def dec_2(fn):
    def inner():
        print("Running dec_1")
        return fn()
    return inner

In [151]:
@dec_1
@dec_2
def my_func():
    print("Running my func")
#stejné jako 
#my_func = dec_1(dec_2(my_func))

Jinými slovy - záleží na pořadí v jakém dekorujeme - tzn. pokud chci posloupnost dekorátorů - musím to stackovat na sobě na základě logiky - první decorator ze shora směrem dolu

In [153]:
my_func() #dec1 - inner, dec2-inner, my_func

Running dec_1
Running dec_1
Running my func


Zatímco pokud :)

In [154]:
def dec_3(fn):
    def inner():
        result = fn()
        print("Running dec_1")
        return result
    return inner

In [155]:
def dec_4(fn):
    def inner():
        result = fn()
        print("Running dec_1")
        return result
    return inner

In [157]:
@dec_3
@dec_4
def my_func2():
    print("Running my func2")

In [158]:
my_func2()

Running my func2
Running dec_1
Running dec_1


Tak dekorátory dodrží pořadí ale funkce má výsledek hned na začátku. Protože nenavracím funkci ale výsledek funkce a tu spouštím předtím.

In [160]:
@dec_1
@dec_1
@dec_1
@dec_1
def my_func3():
    print("Running my func2")

In [161]:
my_func3()

Running dec_1
Running dec_1
Running dec_1
Running dec_1
Running my func2


Jen pro ukázku, mohu stackovat své dekorátory na sebe.